In [1]:
data = readlines("./data/tmp.txt")

x1, x2, y1, y2 = parse.(Int, match(r"target area: x=(\d+)..(\d+), y=([-]\d+)..([-]\d+)", data[1]).captures)

const Point = Tuple{Int,Int}

Base.:+(p1::Point, p2::Point) = (p1[1] + p2[1], p1[2] + p2[2])

struct Rectangle
    corner1::Point
    corner2::Point
end

mutable struct Probe
    position::Point
    velocity::Point
end

function update!(p::Probe)
    p.position += p.velocity
    vx, vy = p.velocity
    p.velocity = ((vx > 0) ? (vx-1) : (vx < 0) ? (vx+1) : 0, vy-1)
end

function launch_probe(vel, target)
    p = Probe((0,0), vel)
    while check(p, target) < 1
        update!(p)
    end
end

target = Rectangle((x1, y1), (x2, y2))

Rectangle(Point(20, -10), Point(30, -5))

In [56]:
# day 20
include("utils.jl")

data = readlines("./data/day-20.txt");

algo = collect(data[1]) .== '#' # light -> true; dark -> false
origimage = permutedims(hcat(collect.(data[3:end])...), [2,1]) .== '#';

function get_padded_image(image, pad_size)
    N = size(image, 1)
    pimage = fill(false, N+2pad_size, N+2pad_size)
    pimage[(pad_size+1):(pad_size+N),(pad_size+1):(pad_size+N)] .= image
    return pimage
end

# window size is 3 and we are planning to apply the algorithm for 2 rounds

# get_pad_size_needed(window_size, nrounds) = nrounds + (window_size-1)

WINSIZE = 3 # window size
PADSIZE = WINSIZE #get_pad_size_needed(WINSIZE, 2)

image = get_padded_image(origimage, PADSIZE)

N = size(image,1)
for step in 1:2
    new_image = fill(false, N, N)
    for i in 2:(N-1)
        for j in 2:(N-1)
            idx = bin2int(vec(image[(i-1):(i+1),(j-1):(j+1)]')) + 1 # +1 due to 1-indexing
            new_image[i,j] = algo[idx]
        end
    end
    if algo[1]
        u = !image[1,1]
        new_image[1,:] .= u
        new_image[end,:] .= u
        new_image[:,1] .= u
        new_image[:,end] .= u
    end
    image = new_image
end

println("Answer 1: ", sum(image))

# 50 rounds
image = get_padded_image(origimage, 51)

N = size(image,1)
for step in 1:50
    new_image = fill(false, N, N)
    for i in 2:(N-1)
        for j in 2:(N-1)
            idx = bin2int(vec(image[(i-1):(i+1),(j-1):(j+1)]')) + 1 # +1 due to 1-indexing
            new_image[i,j] = algo[idx]
        end
    end
    if algo[1]
        u = !image[1,1]
        new_image[1,:] .= u
        new_image[end,:] .= u
        new_image[:,1] .= u
        new_image[:,end] .= u
    end
    image = new_image
end

println("Answer 2: ", sum(new_image))

Answer 1: 5218
Answer 2: 15527


In [96]:
mutable struct Player
    id::Int
    position::Int
    score::Int
    
    Player(id, position, score=0) = new(id, position, score)
end

function update!(p::Player, pos)
    p.position = pos
    p.score += pos
end

mutable struct Die
    sides::Int
    state::Int
    
    Die(sides, state=0) = new(sides, state)
end

function roll!(d::Die)
    rv = d.state+1 # state -> 0-99
    d.state = (d.state + 1) % d.sides
    return rv
end

function multiple_rolls!(d::Die, n)
    [roll!(d) for _ in 1:n]
end

multiple_rolls! (generic function with 1 method)

In [8]:
data = readlines("./data/day-21.txt")

2-element Vector{String}:
 "Player 1 starting position: 6"
 "Player 2 starting position: 3"

In [102]:
poss = [parse(Int, match(r"(\d)$", l).captures[1]) for l in data]

players = [Player(1, poss[1]), Player(2, poss[2])]
die = Die(100)

nrolls = 0
i = 1
while true
    p = players[i]
    r = sum(multiple_rolls!(die, 3))
    nxtpos = ((p.position + r) % 10)
    nxtpos = (nxtpos == 0) ? 10 : nxtpos
    update!(p, nxtpos)
    nrolls += 3
    if p.score >= 1000
        println("Player $i won!")
        break
    end
    # println("Player $i\t$(p.position)\t$(p.score)")
    i = (i == 1) ? 2 : 1
end

println(players)

nrolls * 749

Player 1 won!


In [2]:
TARGETSCORE = 21
TRACKSIZE = 10

struct GameState
    positions::Tuple{Int,Int}
    scores::Tuple{Int,Int}
    turn::Int
end

In [3]:
poss = [parse(Int, match(r"(\d)$", l).captures[1]) for l in data]



LoadError: UndefVarError: data not defined

GameState((4, 8), (0, 0), 1)

In [8]:
mod1idx(a, b) = mod(a-1, b) + 1

function get_next_states(state::GameState)
    i = state.turn # player ID
    p = state.positions[i]
    
    p = p .+ (1:9)
    p = mod1idx.(p, TRACKSIZE)
    
    newpositions = [(i == 1) ? (pi, state.positions[2]) : (state.positions[1], pi) for pi in p]
    newscores = [(i == 1) ? (state.scores[1] + pi, state.scores[2]) : (state.scores[1], state.scores[2] + pi) for pi in p]
    nxtturn = (i == 1) ? 2 : 1
    
    return [GameState(np, ns, nxtturn) for (np, ns) in zip(newpositions, newscores)]
end

get_next_states (generic function with 1 method)

In [28]:
TARGETSCORE = 21
state_dict = Dict{GameState, Vector{GameState}}()
init_state = GameState((4,8), (0, 0), 1)

S = Set([init_state])

step = 0
while !isempty(S)
    state = pop!(S)
    if any(state.scores .≥ TARGETSCORE) && !(state in keys(state_dict))
        state_dict[state] = GameState[]
    elseif !(state in keys(state_dict))
        next_states = filter(s -> !(s in keys(state_dict)), get_next_states(state))
        state_dict[state] = next_states
        union!(S, next_states)
    end
    step += 1
    # println(step, "\t", length(S))
end
println(length(state_dict))

50669


In [29]:
state_dict

Dict{GameState, Vector{GameState}} with 50669 entries:
  GameState((3, 3), (7, 20), 1)   => [GameState((10, 3), (17, 20), 2)]
  GameState((7, 6), (19, 7), 2)   => [GameState((7, 7), (19, 14), 1), GameState…
  GameState((2, 9), (13, 24), 1)  => []
  GameState((8, 9), (11, 14), 1)  => [GameState((9, 9), (20, 14), 2), GameState…
  GameState((5, 1), (12, 3), 1)   => []
  GameState((1, 1), (12, 16), 1)  => [GameState((3, 1), (15, 16), 2), GameState…
  GameState((4, 4), (11, 17), 1)  => [GameState((5, 4), (16, 17), 2), GameState…
  GameState((7, 3), (11, 18), 2)  => [GameState((7, 7), (11, 25), 1), GameState…
  GameState((10, 4), (16, 7), 1)  => [GameState((1, 4), (17, 7), 2), GameState(…
  GameState((4, 6), (18, 20), 1)  => [GameState((5, 6), (23, 20), 2), GameState…
  GameState((5, 1), (16, 6), 1)   => [GameState((6, 1), (22, 6), 2), GameState(…
  GameState((8, 6), (13, 18), 2)  => [GameState((8, 7), (13, 25), 1), GameState…
  GameState((6, 1), (16, 11), 1)  => [GameState((7, 1), (23, 11),

In [30]:
rev_state_dict = Dict(k => GameState[] for k in keys(state_dict))

for (state, nstates) in pairs(state_dict)
    for nstate in nstates
        push!(rev_state_dict[nstate], state)
    end
end

In [31]:
rev_state_dict

Dict{GameState, Vector{GameState}} with 50669 entries:
  GameState((3, 3), (7, 20), 1)  => [GameState((3, 5), (7, 17), 2)]
  GameState((3, 7), (7, 17), 2)  => [GameState((1, 7), (4, 17), 1)]
  GameState((7, 6), (19, 7), 2)  => [GameState((5, 6), (12, 7), 1)]
  GameState((2, 9), (13, 24), 1) => [GameState((2, 6), (13, 15), 2)]
  GameState((7, 8), (26, 15), 2) => [GameState((1, 8), (19, 15), 1)]
  GameState((8, 9), (11, 14), 1) => [GameState((8, 4), (11, 5), 2)]
  GameState((7, 1), (26, 6), 2)  => [GameState((8, 1), (19, 6), 1)]
  GameState((5, 1), (12, 3), 1)  => [GameState((5, 2), (12, 2), 2)]
  GameState((9, 5), (15, 19), 2) => [GameState((1, 5), (6, 19), 1)]
  GameState((2, 6), (7, 18), 1)  => [GameState((2, 9), (7, 12), 2)]
  GameState((8, 7), (11, 13), 2) => [GameState((1, 7), (3, 13), 1)]
  GameState((1, 1), (12, 16), 1) => [GameState((1, 4), (12, 15), 2)]
  GameState((4, 4), (11, 17), 1) => [GameState((4, 8), (11, 13), 2)]
  GameState((5, 6), (18, 11), 2) => [GameState((1, 6), (1